In [1]:
import pandas as pd
import numpy as np

In [12]:
def hurst_exp(input_ts, lags_to_test=150):
    
    tau = []
    lags = []
    
    for lag in range(2, lags_to_test, 1):
        
        lags.append(lag)
        
        # price difference between original series and its lagged part
        price_diff = np.subtract(input_ts[lag:].values, 
                                 input_ts[:-lag].values)
        
        # var is the variance of the price difference series
        var = np.sqrt(np.nanstd(price_diff))
        tau.append(var)
        
    # polyfitting the double-legged series    
    m = np.polyfit(np.log10(lags), np.log10(tau), 1)
    
    hurst = m[0]*2
    
    return hurst

In [13]:
stocklist = pd.read_table("stocklist.txt",names=('A'))

In [14]:
hurst_exp_list = []

for stockname in stocklist['A']:
    
    df_all = pd.read_csv("stock_data/"+stockname+".csv", 
                         names=('Dates', 'stockVWAP', 'futureVWAP', 'bidPrice', 'askPrice', 'total_value', 'total_size'))
    
    df_train = df_all[:60000]
    
    df_train_resampled_10min = df_train[df_train.index % 10 == 0]
    df_train_resampled_10min.reset_index(drop=True)
    df = df_train_resampled_10min[:5999]
    
    hurst_score = hurst_exp(df['stockVWAP'])
    
    hurst_exp_list.append((stockname, hurst_score))

In [15]:
hurst_scores = pd.DataFrame(hurst_exp_list)
hurst_scores = hurst_scores.rename(columns={0:'stockname', 1:'hurst_score'})

In [17]:
selected_stocks = hurst_scores[hurst_scores['hurst_score']<0.5].reset_index(drop=True)

In [18]:
selected_stocks.to_csv('selected_stocks_hurst_exponent.csv')